In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from kernels import *
import pdb
import gpytorch
from itertools import product

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
train_x = torch.linspace(float(0), float(1), int(50))
one = torch.sin(train_x * (float(2) * math.pi)) + torch.randn(train_x.size()) * float(0.2)
two = torch.cos(train_x * (float(2) * math.pi)) + torch.randn(train_x.size()) * float(0.2)
train_y = torch.stack([one, two], int(-1))


In [3]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ZeroMean(), num_tasks=2
        )
        kernel = Diff_SE_kernel()
        kernel2 = Diff_SE_kernel()
        q, dx1, dx2 = var('q, dx1, dx2')
        # TODO test what happens with 
        #L = matrix(2, 2, (dx1, q, 0, dx1))
        # -> does it learn q as a parameter?
        #AND
        #L = matrix(2, 2, (q*dx1, q, 0, dx1))
        # -> does it learn multiple separate q?
        L = matrix(2, 2, (dx1, dx1, 0, dx1))
        R = matrix(2, 2, (dx2, 0, dx2, dx2))
        p = DiffMatrixKernel([[kernel, None], [None, kernel2]])
        self.covar_module = p.diff(left_matrix=L, right_matrix=R)
        #kernel0 = Diff_SE_kernel()
        #kernel1 = Diff_SE_kernel()
        #kernel2 = Diff_SE_kernel()
        #self.covar_module = MatrixKernel([[kernel0, None], [None, kernel2]])

    def forward(self, x):
        #pdb.set_trace()
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        #print(f"{covar_x.detach().evaluate()}")
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x, validate_args=True)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2)
model = MultitaskGPModel(train_x, train_y, likelihood)

In [4]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = int(2) if smoke_test else int(20)


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=float(0.1))  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    for parameter in model.named_parameters():
        print(parameter)
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    #print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f  variance: %.3f noise: %.3f' % (
    #    i + 1, training_iter, loss.item(),
    #    model.covar_module.length.item(),
    #    model.covar_module.var.item(),
    #    model.likelihood.noise.item()
    #))
    optimizer.step()

tensor([[2.0000, 1.0000, 1.9988,  ..., 0.0250, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.9994,  ..., 0.0250, 0.0000, 0.0000],
        [1.9988, 0.9994, 2.0000,  ..., 0.0505, 0.0500, 0.0250],
        ...,
        [0.0250, 0.0250, 0.0505,  ..., 1.0000, 0.9994, 0.9994],
        [0.0000, 0.0000, 0.0500,  ..., 0.9994, 2.0000, 1.0000],
        [0.0000, 0.0000, 0.0250,  ..., 0.9994, 1.0000, 1.0000]],
       grad_fn=<CatBackward>)
tensor([[ 2.2222,  0.9091,  2.2205,  ...,  0.1265, -0.2812,  0.1044],
        [ 0.9091,  1.3580,  0.9086,  ..., -0.1387,  0.1044, -0.1718],
        [ 2.2205,  0.9086,  2.2222,  ...,  0.1490, -0.2270,  0.1265],
        ...,
        [ 0.1265, -0.1387,  0.1490,  ...,  1.3580,  0.9086,  1.3570],
        [-0.2812,  0.1044, -0.2270,  ...,  0.9086,  2.2222,  0.9091],
        [ 0.1044, -0.1718,  0.1265,  ...,  1.3570,  0.9091,  1.3580]],
       grad_fn=<CatBackward>)
tensor([[ 1.9923,  0.8990,  1.9908,  ...,  0.1612, -0.2623,  0.1397],
        [ 0.8990,  1.8344,  0.8986,  .

NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04. Original error on first attempt: cholesky_cpu: U(77,77) is zero, singular U.

In [7]:
for parameter in model.named_parameters():
    print(parameter)

('likelihood.raw_task_noises', Parameter containing:
tensor([-0.9034, -0.9089], requires_grad=True))
('likelihood.raw_noise', Parameter containing:
tensor([-0.9070], requires_grad=True))
('covar_module.kernel_00.kernels.0.var', Parameter containing:
tensor(0.1058, requires_grad=True))
('covar_module.kernel_00.kernels.0.length', Parameter containing:
tensor(0.4735, requires_grad=True))
('covar_module.kernel_00.kernels.1.var', Parameter containing:
tensor(0.1058, requires_grad=True))
('covar_module.kernel_00.kernels.1.length', Parameter containing:
tensor(0.4735, requires_grad=True))
('covar_module.kernel_01.kernels.0.var', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_01.kernels.0.length', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_01.kernels.1.var', Parameter containing:
tensor(1.2556, requires_grad=True))
('covar_module.kernel_01.kernels.1.length', Parameter containing:
tensor(1.0946, requires_grad=True))
('covar_module.ke

In [ ]:
# Set into eval mode
model.eval()
likelihood.eval()

# Initialize plots

number_of_samples = int(50)
# Make predictions
with torch.no_grad():#, gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(float(0), float(2), number_of_samples)
    #pdb.set_trace()
    outputs = model(test_x)
    predictions = likelihood(outputs)
    
    mean = predictions.mean
    lower, upper = predictions.confidence_region()
#print(mean)
#print(lower)
#print(upper)
# This contains predictions for both tasks, flattened out
# The first half of the predictions is for the first task
# The second half is for the second task

#dims = int(2)
#indices = [list(range(i, len(train_y), dims)) for i in range(dims)]


In [ ]:
f, (y1_ax, y2_ax) = plt.subplots(int(1), int(2), figsize=(int(8), int(3)))

# Plot training data as black stars
y1_ax.plot(train_x.detach().numpy(), train_y[:, 0].detach().numpy(), 'k*')
# Predictive mean as blue line
y1_ax.plot(test_x.numpy(), mean[:, 0].numpy(), 'b')
# Shade in confidence
y1_ax.fill_between(test_x.numpy(), lower[:, 0].numpy(), upper[:, 0].numpy(), alpha=0.5)
y1_ax.set_ylim([-3, 8])
y1_ax.legend(['Observed Data', 'Mean', 'Confidence'])
y1_ax.set_title('Observed Values (Likelihood)')

# Plot training data as black stars
y2_ax.plot(train_x.detach().numpy(), train_y[:, 1].detach().numpy(), 'k*')
# Predictive mean as blue line
y2_ax.plot(test_x.numpy(), mean[:, 1].numpy(), 'b')
# Shade in confidence
y2_ax.fill_between(test_x.numpy(), lower[:, 1].numpy(), upper[:, 1].numpy(), alpha=0.5)
y2_ax.set_ylim([-3, 8])
y2_ax.legend(['Observed Data', 'Mean', 'Confidence'])
y2_ax.set_title('Observed Values (Likelihood)')


# Test Diffable SE Kernel

In [9]:
X = torch.tensor([int(1), int(2), int(3)])
X

tensor([1, 2, 3])

In [13]:
x1, x2, l, sigma = var('x1, x2, l, sigma')
lengthscale = 1
variance = 1
SE(x1, x2, l, sigma) = sigma^2*exp(-(x1-x2)^2/(2*l^2))
cov_matr = [[None for i in range(len(X))] for j in range(len(X))]
for i, (v1, v2) in enumerate(product(X, X)):
    cov_matr[int(i/len(X))][int(i%len(X))] = SE.diff(x2).diff(x1)(int(v1), int(v2), lengthscale, variance)
cov_matr

[[1, 0, -3*e^(-2)], [0, 1, 0], [-3*e^(-2), 0, 1]]

In [ ]:
SE.operands()

In [ ]:
a = Diff_SE_kernel(var=int(variance), length=int(lengthscale))
q, dx1, dx2 = var('q, dx1, dx2')
left_poly = dx2
right_poly = dx1
diffed_kernel = a.diff(left_poly=left_poly, right_poly=right_poly, left_d_var=var('dx2'), right_d_var=var('dx1'))
diffed_kernel(X).evaluate()

In [ ]:
def calc_cell_diff(L, M, R, row, col):
    len_M = M.number_of_arguments()
    temp = None
    for j in range(int(sqrt(len_M))):
        if temp == None:
            import itertools
            #M_tr = list(map(list, itertools.zip_longest(*M, fillvalue=None)))
            #[M_tr[j].diff(left_poly=L[row][k], right_poly=R.transpose()[col][j]) for k in range(L.number_of_arguments())]
            temp = L[row]*M.transpose()[j]*R.transpose()[col][j]
        else:
            temp += L[row]*M.transpose()[j]*R.transpose()[col][j]
    return temp

In [ ]:
def diff(self, left_matrix=None, right_matrix=None):
    # iterate left matrix by rows and right matrix by columns and call the
    # respective diff command of the kernels with the row/cols as params
    kernel = MatrixKernel(None)
    output_matrix = [[0 for i in range(np.shape(self.matrix)[1])] for j in range(np.shape(self.matrix)[0])]
    for i, (l, r) in enumerate(itertools.product(left_matrix.rows(), right_matrix.columns())):
        res = self.calc_cell_diff(l, self.matrix, r, context=kernel)
        output_matrix[int(i/np.shape(self.matrix)[0])][
                    int(i % np.shape(self.matrix)[0])]  = res
    kernel.set_matrix(output_matrix)
    return kernel

In [ ]:
def calc_cell_diff(L, M, R, context=None):
    len_M = np.shape(M)[0]
    temp = None
    # https://stackoverflow.com/questions/6473679/transpose-list-
    # of-lists
    M_transpose = list(
       map(list, itertools.zip_longest(*M, fillvalue=None)))
    for r_elem, row_M in zip(R, M_transpose):
        for l_elem, m_elem in zip(L, row_M):
            if temp is None:
                #if M_transpose[int(j/len_M)][j % len_M] is not None:
                if m_elem is not None:
                    temp = l_elem * m_elem*r_elem
                    #temp = l_elem * M_transpose[int(j/len_M)][j % len_M]*r_elem
                else:
                    pass
            else:
                if m_elem is not None:
                #if M_transpose[int(j/len_M)][j % len_M] is not None:
                    temp += l_elem * m_elem*r_elem
                    #temp += l_elem * M_transpose[int(j/len_M)][j % len_M]*r_elem
                else:
                    pass
    return temp.simplify_full()


In [ ]:
dimension = 3
length = dimension*dimension +1
L_list = [var(f'l_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
M_list = [var(f'm_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
R_list = [var(f'r_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
L = matrix(dimension, dimension, L_list)
M = matrix(dimension, dimension, M_list)
R = matrix(dimension, dimension, R_list)
print(L)
print(M)
print(R)
row = 0
col = 0
for row in range(dimension):
    for col in range(dimension):
        print((L*M*R)[row][col])
print("\n\n")
for i, (l, r) in enumerate(itertools.product(L.rows(), R.columns())):

    print(calc_cell_diff(l, M, r))
    print("\n")

In [15]:
SEKernelMatrix = [[sigma^2*exp(-(x1-x2)^2/(2*l^2)), 0], [0, sigma^2*exp(-(x1-x2)^2/(2*l^2))]]
SEKernelMatrix[0][0]


-sigma^2*(x1 - x2)*e^(-1/2*(x1 - x2)^2/l^2)/l^2

In [ ]:
kernel = Diff_SE_kernel()
kernel3 = Diff_SE_kernel()
kernel2 = Diff_SE_kernel()

l = [[kernel, kernel3], [kernel3, kernel2]]
l

In [ ]:
class testobject():
    def __init__(self, val):
        self.val = val
    
    def setVal(self, val):
        self.val = val
        
    def printVal(self):
        return self.val
    
    def __call__(self):
        return self.val

In [ ]:
t1 = testobject(42)
t2 = testobject(21)
t3 = testobject(17)
l = [[t1, t2], [t2, t3]]
print(l)
t2.setVal(170)
print(l[0][1].printVal())
print(l[1][0].printVal())

In [ ]:
q, dx1, dx2 = var('q, dx1, dx2')
left_poly = dx1
right_poly = dx2
L = matrix(2, 2, (dx1, 0, 0, dx1))
R = matrix(2, 2, (dx2, 0, 0, dx2))

In [ ]:
p.diff(left_matrix=L, right_matrix=R).forward(X, X)

In [ ]:
w, q, dx1, dx2 = var('w, q, dx1, dx2')
a = dx1^2
#a.degree(dx1)
a.operands()

In [ ]:
prod([1,2,3])

In [ ]:
a = torch.Tensor([[int(1), int(2), int(3)], [int(4), int(5), int(6)], [int(7), int(8), int(9)]])
for i, row in enumerate(a):
    for j, elem in enumerate(row[i:]):
        print(f"row: {i}, col: {i+j}")
        print(elem)